========================================
   
__Contents__
* Search usage
   1. Import module & Load data
   2. Defining parameter search space
   3. Defining feature search space (optional)
   4. Run search
   
* Log usage
   1. Extract pramater & feature setting
   2. Make meta feature for stacking
   
* Sample:run all backend search

========================================

# Search usage

## 1. Import module & Load data
In here, use the breast cancer wisconsin dataset to modeling.   
This is binary classification dataset.   
Firstly, this dataset is splitted to three datasets(Train, Test)

In [ ]:
import os ,sys
import numpy as np, pandas as pd, scipy as sp
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

from cvopt.model_selection import SimpleoptCV
from cvopt.search_setting import search_category, search_numeric

dataset = datasets.load_breast_cancer()
Xtrain, Xtest, ytrain, ytest = train_test_split(dataset.data, dataset.target, test_size=0.3, random_state=0)

print("Train features shape:", Xtrain.shape)
print("Test features shape:", Xtest.shape)

In [ ]:
from bokeh.io import output_notebook
output_notebook() # When you need search visualization, need run output_notebook()

## 2. Defining parameter search space
Can use a common style in all cv class.

In [ ]:
param_distributions = {
    "penalty": search_category(['none', 'l2']),
    "C": search_numeric(0.01, 3.0, "float"), 
    "tol" : search_numeric(0.0001, 0.001, "float"),  
    "class_weight" : search_category([None, "balanced", {0:0.5, 1:0.1}]),
    }

### 2.A Other styles
Can use other styles depend on base module. 

### for HyperoptCV (base module: Hyperopt)
```python
param_distributions = {
    "penalty": hp.choice("penalty", ['none', 'l2']),
    "C": hp.loguniform("C", 0.01, 3.0), 
    "tol" : hp.loguniform("tol", 0.0001, 0.001), 
    "class_weight" : hp.choice("class_weight", [None, "balanced", {0:0.5, 1:0.1}]),
    }
```
### for  BayesoptCV (base module: GpyOpt)
__NOTE:__
* In GpyOpt, search space is list of dict. But in cvopt, need dict of dict(key:param name, value:dict).
* If `type` is `categorical`,  search space's dict must have `categories` key. `categories` value is category name's list.
```python
param_distributions = {
    "penalty" : {"name": "penalty", "type":"categorical", "domain":(0,1), "categories":['none', 'l2']},
    "C": {"name": "C", "type":"continuous", "domain":(0.01, 3.0)}, 
    "tol" : {"name": "tol", "type":"continuous", "domain":(0.0001, 0.001)}, 
    "class_weight" : {"name": "class_weight", "type":"categorical", "domain":(0,1), "categories":[None, "balanced", {0:0.5, 1:0.1}]},
    }
```

### for  GAoptCV, RandomoptCV
__NOTE:__
* Support search_setting.search_numeric, search_setting.search_category, or scipy.stats class.
```python
param_distributions = {
    "penalty" : hp.choice("penalty", ['none', 'l2']),
    "C": sp.stats.randint(low=0.01, high=3.0), 
    "tol" : sp.stats.uniform(loc=0.0001, scale=0.00009),  
    "class_weight" :  hp.choice("class_weight", [None, "balanced", {0:0.5, 1:0.1}]),
    }
```

## 3. Defining feature search space (optional)
Features are selected per `feature_group`.   
__If `feature_group` is set "-1", this group's features always are used.__   
Criterion of separating group is, for example, random, difference of feature engineering method or difference of data source.
   
When you don't set `feature_group`, optimizer use all input features.

------------------------------------

### Example.   
When data has 5 features(5 cols) and `feature_group` is set as shown below.
   
| feature index(data col index) | feature group |   
|:------------:|:------------:|   
| 0 | 0 |
| 1 | 0 |
| 2 | 0 |
| 3 | 1 |
| 4 | 1 |
   
Define as follows python's list.   
   
```
feature_groups = [0, 0, 0, 1, 1]
```

As search result, you may get flag per `feature_group`.
   
```
feature_groups0: True   
feature_groups1: False
```
   
This result means that optimizer recommend using group 1 features(col index:0,1,2) and not using group 2.

------------------------------------


In [ ]:
feature_groups = np.random.randint(0, 5, Xtrain.shape[1]) 

## 4. Run search
cvopt has API like scikit-learn cross validation class.   
When you had use scikit-learn, you can use cvopt very easy.
   
For each optimizer class's detail, please see [API reference](https://genfifth.github.io/cvopt/).

In [ ]:
estimator = LogisticRegression()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

opt = SimpleoptCV(estimator, param_distributions, 
                 scoring="roc_auc",              # Objective of search
                 cv=cv,                          # Cross validation setting
                 max_iter=32,                    # Number of search
                 n_jobs=3,                       # Number of jobs to run in parallel.
                 verbose=2,                      # 0: don't display status, 1:display status by stdout, 2:display status by graph 
                 logdir="./search_usage",        # If this path is specified, save the log.
                 model_id="search_usage",        # used estimator's dir and file name in save.
                 save_estimator=2,               # estimator save setting.
                 backend="hyperopt",             # hyperopt,bayesopt, gaopt or randomopt.
                 )

opt.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), 
        # validation_data is optional.
        # This data is only used to compute validation score(don't fit).
        # When this data is input & save_estimator=True,the estimator which is fitted whole Xtrain is saved.
        feature_groups=feature_groups, 
        )
ytest_pred = opt.predict(Xtest)

In [ ]:
pd.DataFrame(opt.cv_results_).head() # Search results

# Log usage

## 1. Extract pramater & feature setting
In cvopt, helper function is Included to handle log file easily.   
When you want to extract settings from log file, It can be implemented as follows.

In [ ]:
from cvopt.utils import extract_params
target_index = pd.DataFrame(opt.cv_results_)[pd.DataFrame(opt.cv_results_)["mean_test_score"] == opt.best_score_]["index"].values[0]

estimator_params, feature_params, feature_select_flag  = extract_params(logdir="./search_usage", 
                                                                        model_id="search_usage", 
                                                                        target_index=target_index, 
                                                                        feature_groups=feature_groups)

estimator.set_params(**estimator_params)         # Set estimator parameters
Xtrain_selected = Xtrain[:, feature_select_flag] # Extract selected feature columns

print(estimator)
print("Train features shape:", Xtrain.shape)
print("Train selected features shape:",Xtrain_selected.shape)

## 2. Make meta feature for stacking
When you want to male mete feature and [stacking](https://mlwave.com/kaggle-ensembling-guide/), It can be implemented as follows.   
   
When run search, You need set `save_estimator`>0 to make meta feature.    
In addition, you need set `save_estimator`>1 to make meta feature from the data which is not fitted.

In [ ]:
from cvopt.utils import mk_metafeature
target_index = pd.DataFrame(opt.cv_results_)[pd.DataFrame(opt.cv_results_)["mean_test_score"] == opt.best_score_]["index"].values[0]

Xtrain_meta, Xtest_meta = mk_metafeature(Xtrain, ytrain, 
                                         logdir="./search_usage", 
                                         model_id="search_usage", 
                                         target_index=target_index, 
                                         cv=cv, 
                                         validation_data=(Xtest, ytest), 
                                         feature_groups=feature_groups, 
                                         estimator_method="predict_proba")

print("Train features shape:", Xtrain.shape)
print("Train meta features shape:", Xtrain_meta.shape)
print("Test features shape:", Xtest.shape)
print("Test meta features shape:",  Xtest_meta.shape)

# Sample:run all backend search

In [ ]:
for bk in ["hyperopt", "gaopt", "bayesopt", "randomopt"]:
    estimator = LogisticRegression()
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

    opt = SimpleoptCV(estimator, param_distributions, 
                     scoring="roc_auc",              # Objective of search
                     cv=cv,                          # Cross validation setting
                     max_iter=32,                    # Number of search
                     n_jobs=3,                       # Number of jobs to run in parallel.
                     verbose=2,                      # 0: don't display status, 1:display status by stdout, 2:display status by graph 
                     logdir="./search_usage",        # If this path is specified, save the log.
                     model_id=bk,                    # used estimator's dir and file name in save.
                     save_estimator=2,               # estimator save setting.
                     backend=bk,                     # hyperopt,bayesopt, gaopt or randomopt.
                     )

    opt.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), 
            # validation_data is optional.
            # This data is only used to compute validation score(don't fit).
            # When this data is input & save_estimator=True,the estimator which is fitted whole Xtrain is saved.
            feature_groups=feature_groups, 
            )
    ytest_pred = opt.predict(Xtest)


    from cvopt.utils import extract_params
    target_index = pd.DataFrame(opt.cv_results_)[pd.DataFrame(opt.cv_results_)["mean_test_score"] == opt.best_score_]["index"].values[0]

    estimator_params, feature_params, feature_select_flag  = extract_params(logdir="./search_usage", 
                                                                            model_id=bk, 
                                                                            target_index=target_index, 
                                                                            feature_groups=feature_groups)

    estimator.set_params(**estimator_params)         # Set estimator parameters
    Xtrain_selected = Xtrain[:, feature_select_flag] # Extract selected feature columns

    print(estimator)
    print("Train features shape:", Xtrain.shape)
    print("Train selected features shape:",Xtrain_selected.shape)

    from cvopt.utils import mk_metafeature
    Xtrain_meta, Xtest_meta = mk_metafeature(Xtrain, ytrain, 
                                             logdir="./search_usage", 
                                             model_id=bk, 
                                             target_index=target_index, 
                                             cv=cv, 
                                             validation_data=(Xtest, ytest), 
                                             feature_groups=feature_groups, 
                                             estimator_method="predict_proba")

    print("Train features shape:", Xtrain.shape)
    print("Train meta features shape:", Xtrain_meta.shape)
    print("Test features shape:", Xtest.shape)
    print("Test meta features shape:",  Xtest_meta.shape)